# IMPORTAÇÃO DE BIBLIOTECAS

In [37]:
from skimage import data, io
from scipy import linalg
import matplotlib.pyplot as plt
import chart_studio.tools as tls
import pyvista as pv
import plotly
import plotly.graph_objects as go
import plotly.express as px
import math
import numpy as np
import numpy.matlib
from scipy.sparse import csr_matrix, find
import copy
import stl
from mpl_toolkits import mplot3d
import gmsh

# DETERMINAÇÃO DAS PROPRIEDADES

## PROPRIEDADES DO ARCABOUÇO

In [2]:
#=======================================================================================================
# PCL
#=======================================================================================================
MELT = 5
CFP = 0.3

## PROPRIEDADES DAS CÉLULAS

In [3]:
MELTC = 0.9e12
CFPC = 0.499

## PROPRIEDADES DA ESTRUTURA

In [32]:
EX = size [2]                                                       #NÚMERO DE ELEMENTOS X (PIXELS)
EY = size [1]                                                       #NÚMERO DE ELEMENTOS Y (PIXELS)
EZ = size [0]                                                       #NÚMERO DE ELEMENTOS Z (CAMADAS)
TAM_VOX = 1e-5
L = EX * TAM_VOX                                                    #TAMANHO X
H = EY * TAM_VOX                                                    #TAMANHO Y
W = EZ * TAM_VOX                                                    #TAMANHO Z

# RETIRADA DOS ELEMENTOS FLUTUANTES

## DEFINIÇÃO DAS FUNÇÕES PARA A APLICAÇÃO DO ALGORITMO UNION-FIND

In [4]:
def find(x):
    global parent
    if(parent[x] == x):
        return x
    parent [x] = find(parent[x])
    return parent[x]

def join (x, y):
    global parent
    x = find(x)
    y = find(y)
    if (x == y):
        return
    if (qty[x] <= qty[y]):
        parent[x] = y
        qty [y] += qty [x]
    else:
        parent[y] = x
        qty [x] += qty [y]

## FUNÇÃO PARA A DETERMINAÇÃO DA RELAÇÃO ENTRE A POSIÇÃO NA MATRIZ 3D E O ID DO ELEMENTO

In [5]:
def id(position, size):
    id = position[0] * size [1] * size [2] + position[1] * size [2] + position[2]
    return id

## CARREGAMENTO DO ARQUIVO E VISUALIZAÇÃO

In [18]:
im = io.imread('PCL_corte_retangular_binarizado_diario.tif') #VARIÁVEL "im" É UMA MATRIZ (Z, Y, X)
                                                             #NO CASO, SE A ESPESSURA É 501, O COMPRIMENTO É 546 E A ALTURA É 536,
                                                             #"im" FINAL: (501, 536, 546)
                                                             #VERIFICAR O TAMANHO: "cte.shape"
fig_orig = px.imshow(im, animation_frame=0, labels=dict(animation_frame="Profundidade"), binary_string=True)
fig_orig.show()
size = im.shape
area = size[0]*size[1]*size[2] 

## APLICAÇÃO DO UNION-FIND

In [19]:
global parent
global qty
parent = np.full((area), range(area))
qty = np.full((area), 1)
for z in range(size[0]):
    for y in range(size[1]):
        for x in range (size[2]):
            position = (z, y, x)
            if (im[position] == 255):
                options = [-1, 0, 1]
                for k in options:
                    for j in options:
                        for i in options:
                            if ((i,j,k) == (0,0,0)):
                                continue
                            if ((x+i) >= 0 and (x+i) < size [2] and (y+j) >= 0 and (y+j) < size[1]\
                                and (z+k) >= 0 and (z+k) < size[0]):
                                adj = (z+k, y+j, x+i)
                                if (im[adj] == 255):
                                    join(id(position, size), id(adj, size))

## ELIMINAÇÃO DOS ELEMENTOS FLUTUANTES

Dessa forma, a matriz *im* continua sendo a original e há a criação da *elim_im*, em que os elementos flutuantes foram zerados. Nesse caso ainda criou-se um vetor, denominado *material_elementos*, para armazenar o material (arcabouço ou vazio) constituinte.

In [21]:
material_elementos = np.empty((area + 1), dtype = object)
material_elementos.fill ((0,0))
contador = 0
parent_bgg = np.where(qty == np.amax(qty))
elim_im = copy.deepcopy (im)
for z in range(size[0]):
    for y in range(size[1]):
        for x in range (size[2]):
            position = (z, y, x)
            idty = id(position, size)
            if (find(idty) != parent_bgg and im[position] != 0):
                elim_im [position] = 0
            if elim_im [position] != 0:
                material_elementos [idty + 1] = (MELT, CFP)
elementos_validos = [ i for i in range(len(material_elementos)) if material_elementos[i] != (0,0)]

## VISUALIZAÇÃO APÓS A RETIRADA DOS FLUTUANTES

In [22]:
elim_fig = px.imshow(elim_im, animation_frame=0, labels=dict(animation_frame="Profundidade"),binary_string=True)
elim_fig.show()
io.imsave('resultado.tif', elim_im)

# MAPEAMENTO DAS CÉLULAS

Criação de uma nova matriz, *im_cel*, para a consideração das células como parte do sistema.

In [30]:
im_cel = copy.deepcopy (elim_im)
for z in range(size[0]):
    for y in range(size[1]):
        for x in range (size[2]):
            position = (z, y, x)
            idty = id(position, size)
            if (im_cel[position] == 255):
                options = [-1, 0, 1]
                for k in options:
                    for j in options:
                        for i in options:
                            if ((i,j,k) == (0,0,0) or (i,j) == (-1,1) or (i,j) == (1,1) or \
                                (i,j) == (-1,-1) or (i,j) == (1,-1)):
                                continue
                            if ((x+i) >= 0 and (x+i) < size [2] and (y+j) >= 0 and (y+j) < size[1]\
                                and (z+k) >= 0 and (z+k) < size[0]):
                                adj = (z+k, y+j, x+i)
                                idty_adj = id(adj, size)
                                if (im_cel[adj] == 0):
                                    im_cel [adj] = 128
                                    material_elementos [idty_adj + 1] = (MELTC, CFPC)
elementos_cel = [ i for i in range(len(material_elementos)) if material_elementos[i] == (MELTC, CFPC)]

## VISUALIZAÇÃO APÓS A INSERÇÃO DAS CÉLULAS

In [31]:
fig_cel = px.imshow(im_cel, animation_frame=0, labels=dict(animation_frame="Profundidade"),\
                    color_continuous_scale='gray')
fig_cel.show()

# FORMAÇÃO DAS MATRIZES DE CONECTIVIDADE E DE COORDENADAS

## DETERMINAÇÃO DOS PARÂMETROS BASEADOS NA ESTRUTURA

In [33]:
NX = EX + 1                                                         #NÚMERO DE PONTOS NO EIXO X
NY = EY + 1                                                         #NÚMERO DE PONTOS NO EIXO Y
NZ = EZ + 1                                                         #NÚMERO DE ELEMENTOS NO EIXO Z
NPP = NX * NY                                                       #NÚMERO DE PONTOS POR CAMADA
NEP = EX * EY                                                       #NÚMERO DE ELEMENTOS POR CAMADA
NP = NPP * NZ                                                       #NÚMERO DE PONTOS NO TOTAL
NE = NEP * EZ                                                       #NÚMERO DE ELEMENTOS NO TOTAL

## DETERMINAÇÃO DAS COORDENADAS DOS PONTOS EXISTENTES

In [34]:
X = np.concatenate (([0.0] , np.linspace ( 0 , L , NX )))
Y = np.concatenate (([0.0] , np.linspace ( 0 , H , NY )))
Z = np.concatenate (([0.0] , np.linspace ( 0 , W , NZ )))

## INICIALIZAÇÃO DAS MATRIZES DE CONECTIVIDADE E DE COORDENADAS

In [35]:
mconect = np.full ( ( NE + 1, 8 ),0)
mcoord = np.zeros ( ( NP + 1, 3 ))

## DETERMINAÇÃO DAS MATRIZES DE CONECTIVIDADE E DE COORDENADAS

In [36]:
elemento = 1
linha_elementar = 1
profundidade_elementar = 1
no_atual = 1
while elemento <= NE:
    if elemento == 1:
        #=================================================================================================================
        #PRIMEIRO ELEMENTO DA PRIMEIRA CAMADA
        #=================================================================================================================
        mconect [ elemento, : ] = [ 1 , 2 , 3 , 4 , 5 , 6 , 7, 8 ]

        mcoord [ mconect [ elemento, : ], 0 ] = [ X [ 1 ], X [ 2 ], X [ 2 ], X [ 1 ], X [ 1 ], X [ 2 ], X [ 2 ], X [ 1 ] ]
        mcoord [ mconect [ elemento, : ], 1 ] = [ Y [ 1 ], Y [ 1 ], Y [ 1 ], Y [ 1 ], Y [ 2 ], Y [ 2 ], Y [ 2 ], Y [ 2 ] ]
        mcoord [ mconect [ elemento, : ], 2 ] = [ Z [ 1 ], Z [ 1 ], Z [ 2 ], Z [ 2 ], Z [ 1 ], Z [ 1 ], Z [ 2 ], Z [ 2 ] ]
        no_atual += 7

    elif elemento <= EX:
        #=================================================================================================================
        #PRIMEIRA LINHA DA PRIMEIRA CAMADA
        #=================================================================================================================
        mconect [ elemento ] [ : ] = [ mconect [ elemento - 1, 1 ], no_atual + 1, \
                                      no_atual + 2 , mconect [ elemento - 1, 2 ], \
                                      mconect [elemento - 1, 5 ], no_atual + 3, \
                                      no_atual + 4 , mconect [elemento - 1, 6 ] ]    

        mcoord [ mconect [ elemento, [1, 2, 5, 6] ], 0 ] = [ X [ elemento + 1 ], X [ elemento + 1 ], \
                                                             X [ elemento + 1 ], X [elemento + 1 ] ]
        mcoord [ mconect [ elemento, [1, 2, 5, 6] ], 1 ] = [ Y [ 1 ], Y [ 1 ], Y [ 2 ], Y [ 2 ] ]
        mcoord [ mconect [ elemento, [1, 2, 5, 6] ], 2 ] = [ Z [ 1 ], Z [ 2 ], Z [ 1 ], Z [ 2 ] ]
        no_atual += 4

    elif elemento == ( ( linha_elementar - 1 ) * EX ) + 1 and profundidade_elementar == 1:
        #=================================================================================================================
        #PRIMEIRA COLUNA DA PRIMEIRA CAMADA
        #=================================================================================================================
        mconect [ elemento ] [ : ] = [ mconect [ elemento - EX ] [ 4 ], mconect [ elemento - EX ] [ 5 ], \
                                       mconect [ elemento - EX ] [ 6 ], mconect [ elemento - EX ] [ 7 ],\
                                       no_atual + 1, no_atual + 2, \
                                       no_atual + 3, no_atual + 4]
                                  
        mcoord [ mconect [ elemento, 4:8 ], 0 ] = [ X [ 1 ], X [ 2 ], X [ 2 ], X [ 1 ] ]
        mcoord [ mconect [ elemento, 4:8 ], 1 ] = [ Y [ linha_elementar + 1 ], Y [ linha_elementar + 1 ], \
                                                    Y [ linha_elementar + 1 ], Y [ linha_elementar + 1 ] ]
        mcoord [ mconect [ elemento, 4:8 ], 2 ] = [ Z [ 1 ], Z [ 1 ], Z [ 2 ], Z [ 2 ] ]
        no_atual += 4

    elif profundidade_elementar == 1:
        #=================================================================================================================
        #RESTANTE DA PRIMEIRA CAMADA (O QUE NÃO ESTÁ NEM NA PRIMEIRA COLUNA NEM NA PRIMEIRA LINHA)
        #=================================================================================================================
        mconect [ elemento ] [ : ] = [ mconect [ elemento - EX ] [ 4 ], mconect [ elemento - EX ] [ 5 ], \
                                      mconect [ elemento - EX ] [ 6 ], mconect [ elemento - EX ] [ 7 ],\
                                      mconect [ elemento - 1 ] [ 5 ], no_atual + 1, \
                                      no_atual + 2 , mconect [ elemento - 1 ] [ 6 ] ]

        mcoord [ mconect [ elemento, [5, 6] ], 0 ] = [ mcoord [ mconect [ elemento - EX , 1 ], 0 ], \
                                                       mcoord [ mconect [ elemento - EX , 1 ], 0 ] ]
        mcoord [ mconect [ elemento, [5, 6] ], 1 ] = [ mcoord [ mconect [ elemento - 1 , 5 ], 1 ], \
                                                       mcoord [ mconect [ elemento - 1 , 5 ], 1 ] ]
        mcoord [ mconect [ elemento, [5, 6] ], 2 ] = [ Z [ 1 ], Z [ 2 ] ]
        no_atual += 2

    elif elemento == ( ( profundidade_elementar - 1) * NEP ) + 1:
        #=================================================================================================================
        #PRIMEIRO ELEMENTO DAS CAMADAS EXCETO A PRIMEIRA
        #=================================================================================================================
        mconect [ elemento, : ] = [ mconect [ elemento - NEP ] [ 3 ], mconect [ elemento - NEP ] [ 2 ] ,\
                                    no_atual + 2, no_atual + 1,\
                                    mconect [ elemento - NEP ] [ 7 ], mconect [ elemento - NEP ] [ 6 ],\
                                    no_atual + 3, no_atual + 4 ]
    
        mcoord [ mconect [ elemento, [2, 3, 6, 7] ], 0 ] = [ mcoord [ mconect [ elemento , 1 ], 0 ], \
                                                             mcoord [ mconect [ elemento , 0 ], 0 ],\
                                                             mcoord [ mconect [ elemento , 1 ], 0 ],\
                                                             mcoord [ mconect [ elemento , 0 ], 0 ] ]
        mcoord [ mconect [ elemento, [2, 3, 6, 7] ], 1 ] = [ mcoord [ mconect [ elemento , 1 ], 1 ], \
                                                             mcoord [ mconect [ elemento , 0 ], 1 ],\
                                                             mcoord [ mconect [ elemento , 5 ], 1 ],\
                                                             mcoord [ mconect [ elemento , 4 ], 1 ] ]
        mcoord [ mconect [ elemento, [2, 3, 6, 7] ], 2 ] = [ Z [ profundidade_elementar + 1 ], Z [ profundidade_elementar + 1 ],\
                                                             Z [ profundidade_elementar + 1 ], Z [ profundidade_elementar + 1 ]]
        no_atual += 4

    elif elemento <= ( ( profundidade_elementar -1 ) * NEP ) + EX:
        #=================================================================================================================
        #PRIMEIRA LINHA DAS CAMADAS EXCETO A PRIMEIRA
        #=================================================================================================================
        mconect [ elemento, : ] = [ mconect [ elemento - 1 ] [ 1 ], mconect [ elemento - NEP ] [ 2 ] ,\
                                    no_atual + 1, mconect [ elemento - 1 ] [ 2 ] ,\
                                    mconect [ elemento - 1 ] [ 5 ], mconect [ elemento - NEP ] [ 6 ] ,\
                                    no_atual + 2, mconect [ elemento - 1 ] [ 6 ] ]
    
        mcoord [ mconect [ elemento, [2, 6 ] ], 0 ] = [ mcoord [ mconect [ elemento - NEP , 1 ], 0 ], \
                                                        mcoord [ mconect [ elemento - NEP , 1 ], 0 ] ]
        mcoord [ mconect [ elemento, [2, 6 ] ], 1 ] = [ mcoord [ mconect [ elemento - NEP , 1 ], 1 ], \
                                                        mcoord [ mconect [ elemento - NEP , 5 ], 1 ] ]
        mcoord [ mconect [ elemento, [2, 6 ] ], 2 ] = [ Z [ profundidade_elementar + 1 ], Z [ profundidade_elementar + 1 ] ]
        no_atual += 2

    elif elemento == ( ( linha_elementar - 1 ) * EX ) + 1:
        #=================================================================================================================
        #PRIMEIRA COLUNA DAS CAMADAS EXCETO A PRIMEIRA
        #=================================================================================================================
        mconect [ elemento, : ] = [ mconect [ elemento - EX ] [ 4 ], mconect [ elemento - EX ] [ 5 ] ,\
                                    mconect [ elemento - EX ] [ 6 ], mconect [ elemento - EX ] [ 7 ] ,\
                                    mconect [ elemento - NEP ] [ 7 ], mconect [ elemento - NEP ] [ 6 ] ,\
                                    no_atual + 1, no_atual + 2 ]
    
        mcoord [ mconect [ elemento, 6:8 ], 0 ] = [ mcoord [ mconect [ elemento - NEP , 5 ], 0 ], \
                                                    mcoord [ mconect [ elemento - NEP , 4 ], 0 ] ]
        mcoord [ mconect [ elemento, 6:8 ], 1 ] = [ mcoord [ mconect [ elemento - NEP , 5 ], 1 ], \
                                                    mcoord [ mconect [ elemento - NEP , 4 ], 1 ] ]
        mcoord [ mconect [ elemento, 6:8 ], 2 ] = [ Z [ profundidade_elementar + 1 ], Z [ profundidade_elementar + 1 ] ] 
        no_atual += 2
    else:
        #=================================================================================================================
        #RESTANTE DAS OUTRAS CAMADAS EXCETO A PRIMEIRA (O QUE NÃO ESTÁ NEM NA PRIMEIRA COLUNA NEM NA PRIMEIRA LINHA
        #NEM NA PRIMEIRA CAMADA)
        #=================================================================================================================
        mconect [ elemento ] [ : ] = [ mconect [ elemento - EX, 4 ], mconect [ elemento - EX, 5 ], \
                                      mconect [ elemento - EX, 6 ] , mconect [ elemento - EX, 7 ],\
                                      mconect [ elemento - 1, 5 ], mconect [ elemento - NEP, 6 ],\
                                      no_atual + 1, mconect [ elemento - 1, 6 ] ]

        mcoord [ mconect [ elemento, 6 ], : ] = [ mcoord [ mconect [ elemento - NEP , 5 ], 0 ],\
                                                  mcoord [ mconect [ elemento - NEP , 5 ], 1 ],\
                                                  Z [ profundidade_elementar + 1 ] ]
        no_atual += 1
    if elemento == EX * linha_elementar:
        linha_elementar += 1
    if elemento == NEP * profundidade_elementar:
        profundidade_elementar += 1
    elemento += 1

# SUBDIVISÃO DO ARCABOUÇO EM MAIS ELEMENTOS

## REORGANIZAÇÃO DO ARCABOUÇO EM FACES RETANGULARES

In [38]:
mconect_polydata_square = copy.deepcopy(mconect)[elementos_validos]
mconect_polydata_square = [np.concatenate(([4], face[[0,1,2,3]], [4], face[[0,1,5,4]], [4], face[[1,2,6,5]], \
                           [4], face[[3,2,6,7]], [4], face[[0,3,7,4]], [4], face[[4,5,6,7]] )) \
                           for face in mconect_polydata_square]
faces_square = np.hstack(mconect_polydata_square)

In [44]:
mconect_polydata_square 

[array([  4, 155, 157, 158, 156,   4, 155, 157, 199, 197,   4, 157, 158,
        200, 199,   4, 156, 158, 200, 198,   4, 155, 156, 198, 197,   4,
        197, 199, 200, 198]),
 array([  4, 197, 199, 200, 198,   4, 197, 199, 241, 239,   4, 199, 200,
        242, 241,   4, 198, 200, 242, 240,   4, 197, 198, 240, 239,   4,
        239, 241, 242, 240]),
 array([  4, 199, 201, 202, 200,   4, 199, 201, 243, 241,   4, 201, 202,
        244, 243,   4, 200, 202, 244, 242,   4, 199, 200, 242, 241,   4,
        241, 243, 244, 242]),
 array([  4, 239, 241, 242, 240,   4, 239, 241, 283, 281,   4, 241, 242,
        284, 283,   4, 240, 242, 284, 282,   4, 239, 240, 282, 281,   4,
        281, 283, 284, 282]),
 array([  4, 241, 243, 244, 242,   4, 241, 243, 285, 283,   4, 243, 244,
        286, 285,   4, 242, 244, 286, 284,   4, 241, 242, 284, 283,   4,
        283, 285, 286, 284]),
 array([  4, 281, 283, 284, 282,   4, 281, 283, 325, 323,   4, 283, 284,
        326, 325,   4, 282, 284, 326, 324,   4,

In [42]:
# Initialize gmsh:
gmsh.initialize()
 
# cube points:
lc = 1e-2
point1 = gmsh.model.geo.add_point(0, 0, 0, lc)
point2 = gmsh.model.geo.add_point(1, 0, 0, lc)
point3 = gmsh.model.geo.add_point(1, 1, 0, lc)
point4 = gmsh.model.geo.add_point(0, 1, 0, lc)
point5 = gmsh.model.geo.add_point(0, 1, 1, lc)
point6 = gmsh.model.geo.add_point(0, 0, 1, lc)
point7 = gmsh.model.geo.add_point(1, 0, 1, lc)
point8 = gmsh.model.geo.add_point(1, 1, 1, lc)
 
# Create the relevant Gmsh data structures
# from Gmsh model.
gmsh.model.geo.synchronize()
 
# Generate mesh:
gmsh.model.mesh.generate()
 
# Write mesh data:
gmsh.write("GFG.msh")
 
gmsh.fltk.run()
 
# It finalize the Gmsh API
gmsh.finalize()